In [19]:
!pip install transformers datasets 


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
!pip install rouge.score nltk py7zr


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0


In [22]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\abulf\appdata\local\temp\pip-req-build-v_523uiy
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4320 sha256=63cc2c90318a70dd9de88e8d7a2f0fe4989a2f36cd7bcf18b4d4dbf5c210c763
  Stored in directory: C:\Users\abulf\AppData\Local\Temp\pip-ephem-wheel-cache-u015_rqn\wheels\a8\b9\18\23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\abulf\AppData\Local\Temp\pip-req-build-v_523uiy'

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
%load_ext nvcc_plugin

created output directory at c:\Users\abulf\Documents\Python_tutorial\Assignments\Machine_learning\src
Out bin c:\Users\abulf\Documents\Python_tutorial\Assignments\Machine_learning\result.out


In [24]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abulf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
data = load_dataset('joelito/legal_case_document_summarization')
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-cnn'

Found cached dataset json (C:/Users/abulf/.cache/huggingface/datasets/joelito___json/joelito--legal_case_document_summarization-f8fdb1d237cf12fe/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00,  7.17it/s]


In [26]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [27]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['judgement']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [28]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['dataset_name', 'judgement', 'summary'])

Loading cached processed dataset at C:\Users\abulf\.cache\huggingface\datasets\joelito___json\joelito--legal_case_document_summarization-f8fdb1d237cf12fe\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-8203440fae79c91a.arrow
Loading cached processed dataset at C:\Users\abulf\.cache\huggingface\datasets\joelito___json\joelito--legal_case_document_summarization-f8fdb1d237cf12fe\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-4962dae76082c076.arrow


In [29]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7773
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [30]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [31]:
batch_size = 10

In [32]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [33]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [34]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    # learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=30,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True  #for cuda not available
    )

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['test'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.575300,2.030761,48.565000,23.638000,32.264900,43.122900,129.650000
2,1.358700,2.107896,49.259200,24.380600,32.883000,43.681800,129.195000
2,1.139200,2.186799,48.593100,23.401000,31.985700,43.147900,129.130000
4,0.890700,2.413578,48.772700,23.728800,32.160200,42.916100,128.960000


In [ ]:
# model.save_pretrained('legalBart')

In [ ]:
# tokenizer.save_pretrained('tokenlegalBart')

('tokenlegalBart/tokenizer_config.json',
 'tokenlegalBart/special_tokens_map.json',
 'tokenlegalBart/vocab.json',
 'tokenlegalBart/merges.txt',
 'tokenlegalBart/added_tokens.json',
 'tokenlegalBart/tokenizer.json')